In [9]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

# Inicializa el entorno AMPL con los módulos que necesitas y la licencia UUID
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"],  # Selección de módulos
    license_uuid="bb2c71f6-2b4c-4570-8eaf-4db0b0d7349a"  # Tu licencia UUID
)


Note: you may need to restart the kernel to use updated packages.
Licensed to Bundle #6733.7184 expiring 20241231: ICI5142-01 INVESTIGACION DE OPERACIONES AVANZADAS (Advanced Operations Research), Prof. Guillermo Cabrera-Guerrero, Pontificia Universidad Catolica de Valparaiso.


In [10]:
%%writefile cflp.mod
# Definición de conjuntos
set SR;  # Conjunto de regiones proveedoras
set DR;  # Conjunto de regiones demandantes

# Definición de parámetros
param fixcost{SR};    # Costo fijo para abrir una planta en cada región proveedora
param capacity{SR};   # Capacidad de las plantas en las regiones proveedoras
param demand{DR};     # Demanda en las regiones demandantes
param transp_cost{SR, DR};  # Costo de transporte entre regiones proveedoras y demandantes

# Variables de decisión
var Q{SR, DR} >= 0;  # Cantidad transportada de la región i a la región j
var y{SR} binary;    # Decisión de abrir o no la planta en la región i

# Función objetivo: minimizar el costo total
minimize Totalcost:
    sum{i in SR, j in DR} Q[i,j] * transp_cost[i,j] +
    sum{i in SR} y[i] * fixcost[i];

# Restricciones

# Satisfacer la demanda en todas las regiones demandantes
subject to Demand_Satisfaction{j in DR}:
    sum{i in SR} Q[i,j] == demand[j];

# No exceder la capacidad de las plantas abiertas
subject to Capacity_Limit{i in SR}:
    sum{j in DR} Q[i,j] <= capacity[i] * y[i];

Overwriting cflp.mod


In [11]:
%%writefile Mcflp.mod
# Definición de conjuntos
set SR;  # Conjunto de regiones proveedoras
set DR;  # Conjunto de regiones demandantes

# Definición de parámetros
param fixcost{SR};    # Costo fijo para abrir una planta en cada región proveedora
param capacity{SR};   # Capacidad de las plantas en las regiones proveedoras
param demand{DR};     # Demanda en las regiones demandantes
param transp_cost{SR, DR};  # Costo de transporte entre regiones proveedoras y demandantes

# Variables de decisión
var Q{SR, DR} >= 0;  # Cantidad transportada de la región proveedora i a la región demandante j
var y{SR} binary;    # Decisión de abrir o no la planta en la región i

# Función objetivo: minimizar el costo total
minimize Totalcost:
    sum{i in SR, j in DR} Q[i,j] * transp_cost[i,j] +
    sum{i in SR} y[i] * fixcost[i];

# Restricciones

# Satisfacer la demanda en todas las regiones demandantes
subject to Demand_Satisfaction{j in DR}:
    sum{i in SR} Q[i,j] >= demand[j];  # Se permite que una región demandante sea abastecida por varias plantas

# No exceder la capacidad de las plantas abiertas
subject to Capacity_Limit{i in SR}:
    sum{j in DR} Q[i,j] <= capacity[i] * y[i];

# Restringir las cantidades transportadas solo desde plantas abiertas
subject to Supply_Only_From_Open_Plants{i in SR, j in DR}:
    Q[i,j] <= capacity[i] * y[i];


Overwriting Mcflp.mod


In [12]:
%%writefile cflp.dat
# Conjuntos
set SR := 1 2 3;  # Regiones proveedoras
set DR := A B;    # Regiones demandantes

# Parámetros
param fixcost :=
    1 1000
    2 1500
    3 1200;

param capacity :=
    1 500
    2 700
    3 600;

param demand :=
    A 300
    B 400;

param transp_cost :  A  B :=
    1   4   6
    2   2   3
    3   5   8 ;


Overwriting cflp.dat


In [13]:
%%writefile Mcflp.dat
# Conjuntos
set SR := 1 2 3 4;  # Regiones proveedoras
set DR := A B C;    # Regiones demandantes

# Parámetros
param fixcost :=
    1 1200
    2 1600
    3 1300
    4 1800;

param capacity :=
    1 500
    2 800
    3 600
    4 700;

param demand :=
    A 350
    B 450
    C 300;

param transp_cost :  A  B  C :=
    1   4   6   5
    2   3   2   7
    3   6   5   4
    4   7   3   6 ;


Overwriting Mcflp.dat


In [14]:
%%ampl_eval
model cflp.mod;
data cflp.dat;
option solver gurobi;
solve;
display Q,y;

Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 3300
2 simplex iterations
1 branching node
Q :=
1 A     0
1 B     0
2 A   300
2 B   400
3 A     0
3 B     0
;

y [*] :=
1  0
2  1
3  0
;



In [15]:
%%ampl_eval
reset;
model Mcflp.mod;
data Mcflp.dat;
option solver gurobi;
solve;
display Q, y;


Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 6050
8 simplex iterations
1 branching node
Q :=
1 A     0
1 B     0
1 C     0
2 A   350
2 B   450
2 C     0
3 A     0
3 B     0
3 C   300
4 A     0
4 B     0
4 C     0
;

y [*] :=
1  0
2  1
3  1
4  0
;



Inicialización: Se crea una población inicial de individuos aleatorios.
Evaluación: Se evalúa cada individuo utilizando la función de aptitud.
Selección: Se seleccionan los individuos más aptos para reproducirse.
Cruce: Los individuos seleccionados se combinan para formar nuevos individuos.
Mutación: Se mutan algunos individuos aleatoriamente para garantizar la diversidad.
Reemplazo: Los individuos de la nueva generación reemplazan a la población anterior.

Repetición: Los pasos del 2 al 6 se repiten hasta que se cumpla la condición de parada.

In [22]:
import numpy as np



def algoritmo_genetico(filas,columnas):
   matriz_random = np.random.random((filas, columnas))
   print(matriz_random)


filas = 4  
columnas = 5  

algoritmo_genetico(filas,columnas)



In [16]:
import random

# Suponiendo que F es el número de instalaciones y C el número de clientes
F = 10  # Número de instalaciones
C = 100  # Número de clientes

# Generar apertura aleatoria de instalaciones
instalaciones_abiertas = [random.choice([0, 1]) for _ in range(F)]

# Asignar clientes a las instalaciones abiertas
asignaciones = [random.choice([i for i in range(F) if instalaciones_abiertas[i] == 1]) for _ in range(C)]
print(asignaciones)

[3, 8, 6, 2, 9, 6, 2, 6, 9, 9, 1, 1, 8, 0, 8, 3, 1, 2, 2, 2, 6, 0, 8, 8, 6, 9, 3, 6, 8, 3, 1, 2, 9, 2, 1, 6, 3, 9, 8, 9, 1, 1, 0, 0, 8, 0, 6, 8, 2, 9, 8, 2, 6, 3, 6, 9, 9, 9, 3, 8, 0, 3, 6, 1, 2, 6, 2, 9, 2, 1, 0, 1, 6, 2, 2, 6, 1, 2, 6, 6, 3, 3, 0, 2, 0, 6, 8, 8, 6, 6, 1, 0, 1, 3, 1, 8, 0, 2, 1, 6]
